Napari Viewer

Author: Pablo Siliceo Portugal (psiliceop@gmail.com)

Downloading NAPARI

In [ ]:
#Downloading Napari for the first time
conda create -y -n napari-env -c conda-forge python=3.9
conda activate napari-env
conda install -c conda-forge napari

#****Napari can also be installed with pip using
# python -m install "napari[all]"

Dependencies needed

In [2]:
try:
    import napari
    import pandas
    from napari.layers import Shapes
    from napari.utils.notifications import show_info
except:
    pass
import pandas as pd
import numpy as np
import random
import tifffile as tiff
from tifffile import imread
import dask.array as da
from dask.cache import Cache
import zarr
import os
import matplotlib.patches as mpatches
import scipy.spatial.distance as sdistance
import ast
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure
from pathlib import Path
from magicgui import event_loop, magicgui
from PyQt5.QtWidgets import QMessageBox
import sys
from PyQt5.QtWidgets import QApplication, QPushButton
from PyQt5.QtCore import Slot
import enum
from dask_image.imread import imread as daskread
#from joblib import Parallel, delayed

In [4]:
#This code will open the Napari viewer with the necesary widgets to open and perform operations on the image 
viewer = napari.Viewer()

#Widgets
#Opening the image in the viewer
@magicgui(call_button='Open image', layout='vertical')
def open_large_image(image_path=Path(), contrast_limit_txt=Path(), ab_list_path = Path()):
    ab = pd.read_csv(ab_list_path)
    ab = list(ab["ABS"])
    cl_txt = str(contrast_limit_txt)
    
    if cl_txt =='.':
    
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0, name = ab, visible=False)

    else:
        cl = open(cl_txt, 'r')
        c = cl.read()
        contrast_limit = ast.literal_eval(c)
        
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0, name = ab, visible=False,contrast_limits=contrast_limit)
        cl.close()

#Open the image segmentation mask (if any) in the viewer
@magicgui(call_button='Open mask', layout='vertical')
def open_mask(mask_path=Path()):
    seg_m = tiff.imread(mask_path)
    if (len(seg_m.shape) > 2) and (seg_m.shape[0] > 1):
        seg_m = seg_m[0]
    viewer.add_labels(seg_m, name='MASK')

#Loading previously created shapes in hte viewer        
@magicgui(call_button = 'Load Shapes', layout='vertical', shapes_path={"mode": "w", "mode": "d"})
def load_shapes(shapes_path: Path):
    shapes_path = str(shapes_path) + "/"
    shapes_list = os.listdir(shapes_path)
    names = list()
    for n in range(0,len(shapes_list)):
        names.append(shapes_list[n].replace(".txt",""))
        locals()[str(names[n])] = list()
    for i in range(0,len(shapes_list)):
        shapes = open(shapes_path + shapes_list[i], 'r')
        shapes = shapes.read()
        shapes = shapes.replace('\n', '').replace('      ','').replace('array(','').replace(')','')
        shapes = ast.literal_eval(shapes)
        for x in range(0,len(shapes)):
            locals()[str(names[i])].append(np.array(shapes[x]))
            shapes_layer = viewer.add_shapes(locals()[str(names[i])], shape_type='polygon', edge_width=0, 
                                     edge_color='#777777ff', face_color='white', name= names[i])
            
#Saving contrast limits done in the image       
@magicgui(call_button='Save contrast limits', layout='vertical', output_file={"mode": "w", "mode": "d"})
def save_contrast_limits(output_file: Path, ab_list_path = Path(), name = ""):
    contrast_limit = list()
    ab = pd.read_csv(ab_list_path)
    ab = list(ab["ABS"])
    for antibody in ab:
        contrast_limit.append(viewer.layers[antibody].contrast_limits)

    with open(str(output_file) + "/" + name + ".txt" , "w") as output:
        output.write(str(contrast_limit))

#Saving a shape done in the image (array of coordenates as a .txt file)    
@magicgui(call_button='Save shape array', layout='vertical', output_file={"mode": "w", "mode": "d"})
def save_shapes(output_file : Path, shape_name = ""):
    shapes = viewer.layers[shape_name].data
    with open(str(output_file) + "/" + shape_name + ".txt", 'w') as output:
        output.write(str(shapes))
        
#Obtaining a csv file of the cell IDs from a selected shape
@magicgui(call_button='Cut and Save ROIs', filepath={"mode": "w", "mode": "d"})
def cut_mask(filepath: Path, shape_name = ""):
    mask_to_cut = viewer.layers.selection.active.data
    newmask = np.copy(mask_to_cut)
    shape = viewer.layers.selection.active.data.shape
    selected_area = viewer.layers[shape_name].to_labels(labels_shape=shape)
    removable_cells =[]
    for i in range(0,mask_to_cut.shape[0]):
        for j in range(0,mask_to_cut.shape[1]):
            cell = mask_to_cut[i,j]
            if selected_area[i,j] > 0 and cell not in removable_cells and cell > 0:
                removable_cells.append(cell)
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + '/' + shape_name + '_selected_cell_ids.csv', index=False)


viewer.window.add_dock_widget(open_large_image)
viewer.window.add_dock_widget(open_mask)
viewer.window.add_dock_widget(load_shapes)
viewer.window.add_dock_widget(save_contrast_limits)
viewer.window.add_dock_widget(save_shapes)
viewer.window.add_dock_widget(cut_mask)